In [ ]:
"""Pipelin

Here we design and define the pipeline for the project.

Structure:
    1. Imports, Variables, Functions
    2. Load Data
    3. Preprocess Data
    4. Train Model
    5. Evaluate Model
    6. Save Model
"""

# 1. Imports, Variables, Functions
# imports
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns

# variables


# functions

# 2. Load Data
# 3. Preprocess Data
# 4. Train Model
# 5. Evaluate Model
# 6. Save Model